In [ ]:
#import matplotlib.pyplot as plt
#import pandas

#df = pandas.read_csv('./mnist_train.csv',header=None)   #这个MNIST的每一行包含785个值，第一个表示数字，其余的784个值是图像（28×28像素）的像素值
#df.head()  #显示前5行
#df.info()  #显示dataframe的行、列数等信息

#选取手写数字数据集的一行，可视化：
#data = df.iloc[0] #选择数据集的第零行，赋给data,是一个两列的数组，第一列是key，第二列是value
#label = data[0]
#img=data[1:].values.reshape(28,28)
#plt.title("label = " + str(label))
#plt.imshow(img, interpolation='none',cmap='Blues')  #第二个参数是指无须平滑像素，第三个参数是指定调色板的颜色为蓝色
#plt.show()

#使用S型逻辑函数Sigmoid

In [ ]:
import torch
import torch.nn as nn
import pandas

#构建神经网络类（继承父类）：
class Classifier(nn.Module):
    def __init__(self):
        #初始化PyTorch的父类
        super().__init__()
    
        #定义神经网络层
        self.model=nn.Sequential(     #nn.Sequential()允许我们提供一个网络模块的列表
            nn.Linear(784,200),       #从输入层到中间层的全连接映射，包含链接权重
            #nn.Sigmoid(),             #激活函数，把中间层的输入变为中间层的输出
            nn.LeakyReLU(0.02),       #这个激活函数更好一些，0.02是函数左半边的梯度
            
            nn.LayerNorm(200),        #用于标准化

            nn.Linear(200,10),        #中间层到输出层的全连接
            nn.Sigmoid()              #激活函数，得到最终输出
            #nn.LeakyReLU(0.02)
        ) #定义的是正向传导

        #定义误差
        #self.loss_function = nn.MSELoss()   #这个是均方误差，先计算每个输出节点的实际输出和预期输出之差的平方，再计算平均值，测试用例的分数约为87%
        self.loss_function = nn.BCELoss()    #这个是二元交叉熵损失，后面有说，测试用例的分数为90.28%
        
        #定义更新权重的方法，即定义优化器，这里用的是随机梯度下降SGD，学习率为0.01
        #所有的可学习参数（权重、bias）可以通过self.parameters()访问
        #self.optimiser = torch.optim.SGD(self.parameters(), lr=0.01)
        self.optimiser = torch.optim.Adam(self.parameters())             #这个更新权重的方法更好

        #用于跟踪训练
        self.counter = 0
        self.progress = []

    #这个函数用inputs输入网络，通过正向传导返回输出
    def forward(self, inputs):
        #直接运行模型
        return self.model(inputs)    #self.model()由nn.Sequential()定义，调用这个函数直接得到神经网络的输出

    #首先需要训练网络：
    #train()既需要网络的输入值，也需要预期的目标值，以计算损失
    def train(self, inputs, targets):
        #计算网络的输出值
        outputs = self.forward(inputs)
        #计算损失值
        loss = self.loss_function(outputs, targets)     #看一下误差函数的用法
    
        #接下来使用损失更新链接权重：
        self.optimiser.zero_grad()  #梯度归零：每次训练之前要先将梯度归零
        loss.backward()             #反向传播
        self.optimiser.step()       #更新权重：逐步（step）沿着梯度更新可学习参数

        #到此神经网络主体差不多了
        #接下来是可视化训练，即跟踪训练，一种方法是监控损失

        #每隔10个训练样本增加一次计数器的值，将损失值添加到列表末尾
        self.counter += 1
        if(self.counter % 10 == 0):
            self.progress.append(loss.item())   #这里的item()只是为了方便展开一个单值张量
            
        if(self.counter % 10000 == 0):
            print("counter = ",self.counter)
    
    #将损失值绘制成图：
    def plot_progress(self):
        #将损失值列表转换为一个pandas DataFrame
        df = pandas.DataFrame(self.progress, columns=['loss'])

        #用plot()函数，并调整图的设计和风格 
        df = df.astype(float)
        df.plot(ylim=(0, 1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5))


In [ ]:
#以PyTorch的方式加载和使用数据：
#导入PyTorch的torch.utils.data.Dataset类：
from torch.utils.data import Dataset
import pandas
import matplotlib.pyplot as plt

#构建一个MnistDataset类，继承Dataset类：
class MnistDataset(Dataset):
    def __init__(self, csv_file):
        self.data_df = pandas.read_csv(csv_file, header=None)    #读入csv数据，其实只有这个最有用，其他三个函数在训练中没啥用
    
    def __len__(self):                #返回DataFrame的大小
        return len(self.data_df)
    
    def __getitem__(self, index):     #从数据集中第index项中提取一个标签,用这个类实例化后，xxx[index]可以调用这个函数直接返回三项信息
        #目标图像、标签
        label = self.data_df.iloc[index,0]    #选择第index行，第0列的数据，是个数组
        target = torch.zeros((10))            #创建一个维度为10的张量变量target表示神经网络的预期输出
        target[label] = 1.0                   #除了与标签对应项是1，其余为0

        #图像数据，取值范围为0~255，标准化为0~1
        image_values = torch.FloatTensor(self.data_df.iloc[index,1:].values) / 255.0
        #.iloc[index,1:]解释：选择第index行、从第一列到最后一列的数据
        #df.iloc[1:3, 2:4]是指读取第1、2行，第2、3列的数据
        #注意！行、列都从0开始编号！ 

        #返回标签、图像数据张量以及目标张量
        return label, image_values, target
    
    #添加一个制图方法：
    def plot_image(self, index):
        arr = self.data_df.iloc[index,1:].values.reshape(28,28)
        plt.title("label = " + str(self.data_df.iloc[index,0]))
        plt.imshow(arr, interpolation='none', cmap='Blues')
        pass
    

In [ ]:
%%time
#notebook的魔法指令，可以计时只能放在第一行
#开始训练：

mnist_dataset = MnistDataset('./mnist_train.csv')

#创建神经网络
C = Classifier()

#在训练循环周期添加一个外部周期循环来多次使用整个数据集：
epochs = 3
for i in range(epochs):
    print('training epoch', i+1, "of", epochs)
    for label, image_data_tensor, target_tensor in mnist_dataset:    #对每个样本项，传给train()
        C.train(image_data_tensor, target_tensor)
        pass
    pass

C.plot_progress()

In [ ]:
#现在测试数据：
mnist_test_dataset = MnistDataset('./mnist_test.csv')

#随便挑选一副图像
record = 258

#绘制图像和标签
#mnist_test_dataset.plot_image(record)

image_data = mnist_test_dataset[record][1]

#调用训练后的神经网络
output = C.forward(image_data)
maxelem = torch.max(output)
#绘制输出张量
#输出被转化为一个简单的numpy数组，再被包装为一个DataFrame，绘制柱形图
#detach()方法来创建一个新的张量，该张量与原始张量具有相同的值，但不再与计算图相关联。然后，如果需要将该张量转换为NumPy数组，可以使用numpy()方法。
pandas.DataFrame(output.detach().numpy()).plot(kind='bar',legend=False,ylim=(0,1))

#输出中的值谁最大就被认为是最可能的，即为结果

In [ ]:
#测试用训练数据训练后的网络

score=0   #匹配则加一
items=0   #计数总的测试数

for label, image_data_tensor, target_tensor in mnist_test_dataset:
    answer = C.forward(image_data_tensor).detach().numpy()
    
    #numpy数组的.argmax()的作用是输出最大值的索引
    if (answer.argmax() == label):
        score += 1
        pass
    items += 1
    pass

print(score, items, score/items)

#3个外周期，即epochs=3，分数约为87%
#1个外周期，即epochs=1，分数约为60%

In [ ]:
#改良：
#有时会把一个预测温度的网络输出调到0~100的任何值，有时会把网络设计成输出true/false或1/0，例如判断一副图画是不是猫
#均方误差只适用于回归任务，不太适用于分类任务
#可替代为二元交叉熵损失，它同时惩罚置信度高的错误输出和置信值低的正确输出，叫：nn.BCELoss()

#注意！！nn.BCELoss()和nn.LeakyReLU(0.02)一起用会报错？？？？
